**Train on whole dataset and test on test set**

In [1]:
import os
import pandas as pd 
import numpy as np

from expert_tree import get_expert_tree_results, Expert_Tree
from wrapper import best_first_search_mg

from sklearn.metrics import confusion_matrix

Test define feature subset

In [2]:
# feats1 = ['ArmSwelling', 'FHT', 'BreastSwelling', 'Skin', 'DISCOMFORT', 
#          'TIME_LAPSE_LOG', 'Mobility', 'PAS', 'BMI', 
#          'ChestWallSwelling', 'Chemotherapy', 'Mastectomy']

# feats2 = ['ArmSwelling', 'SYM_COUNT', 'FHT', 'BreastSwelling', 'Skin', 'DISCOMFORT', 
#          'TIME_LAPSE_LOG', 'Mobility', 'PAS']

# feats3 = ['ArmSwelling', 'FHT', 'BreastSwelling', 'Skin', 'DISCOMFORT', 
#          'TIME_LAPSE_LOG', 'PAS', 'BMI', 
#          'SLNB_Removed_LN', 'Radiation', 'Lumpectomy', 'Chemotherapy']

# feats4 = ['ArmSwelling', 'BreastSwelling', 'Skin', 'PAS', 'DISCOMFORT', 'Age', 
#          'SLNB_Removed_LN', 'BMI', 'TIME_LAPSE_LOG']

# feat5 = ['ArmSwelling', 'SYM_COUNT', 'BreastSwelling', 'TIME_LAPSE_LOG', 'Age', 'Skin',
#  'ALND_Removed_LN', 'SLNB_Removed_LN']

# feat6 = ['ArmSwelling', 'SYM_COUNT', 'BreastSwelling', 'TIME_LAPSE_LOG', 'FHT', 'Age', 'Skin',
#         'DISCOMFORT', 'Lumpectomy']

# feat7 = ['ArmSwelling', 'BreastSwelling', 'Skin', 'PAS', 'FHT', 'DISCOMFORT',
#          'SYM_COUNT', 'Chemotherapy', 'Age', 'TIME_LAPSE_LOG']

# feat8 = ['ArmSwelling', 'BreastSwelling', 'Skin', 'PAS', 'FHT', 'DISCOMFORT',
#  'SYM_COUNT', 'ALND_Removed_LN', 'Lumpectomy', 'TIME_LAPSE_LOG']

# feat9 = ['Mobility', 'ArmSwelling', 'BreastSwelling', 'Skin', 'FHT', 'SYM_COUNT',
#  'ChestWallSwelling', 'Radiation', 'Age', 'TIME_LAPSE_LOG']

feat_gbt = ['Mobility', 'ArmSwelling', 'BreastSwelling', 'Skin', 'FHT', 'SYM_COUNT', 'Age', 'TIME_LAPSE_LOG']
feat_gbt_without_log = ['Mobility', 'ArmSwelling', 'BreastSwelling', 'Skin', 'FHT', 'SYM_COUNT', 'Age', 'TIME_LAPSE']


feat_gbt_backward = ['Mobility', 'ArmSwelling', 'BreastSwelling', 'Skin', 'SYM_COUNT',
                     'Chemotherapy', 'Radiation', 'Age', 'Lumpectomy', 'TIME_LAPSE_LOG']

# cart_feat1 = ['ArmSwelling', 'BreastSwelling', 'Skin', 'SYM_COUNT', 'Chemotherapy', 'Hormonal']

# cart_feat2 = ['SYM_COUNT', 'ArmSwelling', 'BreastSwelling', 'TIME_LAPSE_LOG', 'Skin', 'Age']

# cart_feat3 = ['Mobility', 'ArmSwelling', 'BreastSwelling', 'DISCOMFORT', 'SYM_COUNT','Radiation', 'SLNB_Removed_LN', 'Lumpectomy', 'Hormonal', 'TIME_LAPSE_LOG']

# cart_feat4 = ['ArmSwelling', 'BreastSwelling', 'DISCOMFORT', 'SYM_COUNT', 'Radiation'
#               ,'SLNB_Removed_LN', 'Mastectomy', 'Lumpectomy', 'Hormonal', 'TIME_LAPSE_LOG']

feat_cart = ['SYM_COUNT', 'ArmSwelling', 'BreastSwelling', 'TIME_LAPSE_LOG', 'DISCOMFORT']

feat_cart_backward = ['Mobility', 'ArmSwelling', 'BreastSwelling', 'DISCOMFORT', 'SYM_COUNT',
                      'Radiation', 'SLNB_Removed_LN', 'Mastectomy', 'Hormonal', 'TIME_LAPSE_LOG']

In [3]:
feat_gbt_forward_binary = ['ArmSwelling','BreastSwelling', 'PAS', 'SYM_COUNT', 'Age', 'BMI', 'TIME_LAPSE_LOG']
feat_gbt_backward_binary = ['Mobility', 'ArmSwelling', 'BreastSwelling', 'Skin', 'PAS', 'DISCOMFORT',
 'SYM_COUNT', 'ChestWallSwelling', 'Chemotherapy', 'Radiation', 'Age',
 'SLNB_Removed_LN', 'Mastectomy', 'Lumpectomy', 'Hormonal', 'BMI',
 'TIME_LAPSE_LOG']

In [4]:
num_class = 3
if num_class == 2:
    w = [0.45258215, 1-0.45258215]
else:
    w = [0.15913545, 0.54254953, 0.29831502]
print(w)

feats = feat_gbt
if_all_feature = False
if if_all_feature:
    feats = data_train.columns.values[:-1]

[0.15913545, 0.54254953, 0.29831502]


In [5]:
estimator_types = ['gbt', 'cart']
estimator_type = estimator_types[0]
print('estimator_type:', estimator_type)

estimator_type: gbt


In [6]:
data_train = pd.read_csv('./data/result_data/split_train_Oct21_stratified.csv')
#data_test = pd.read_csv('./data/result_data/split_test_Oct21_stratified.csv')
data_test = pd.read_csv('./data/web_data_ML_correct.csv')
overlap = set(data_train['Username'].values) & set(data_test['Username'].values)
print(overlap)
assert len(overlap) == 0, "there is overlap between training and testing!!!"
data_train = data_train.drop(columns=['Unnamed: 0', 'Username'])
#data_test = data_test.drop(columns=['Unnamed: 0', 'Username'])

set()


In [7]:
X_train, Y_train = data_train[feats], data_train.iloc[:,-1]
print('column:', X_train.columns.values)
X_train, Y_train = X_train.values, Y_train.values


column: ['Mobility' 'ArmSwelling' 'BreastSwelling' 'Skin' 'FHT' 'SYM_COUNT' 'Age'
 'TIME_LAPSE_LOG']


Pre process web data

In [8]:
# drop data within a 6 months
data_test = data_test[data_test.TIME_LAPSE >=0.5]
data_test= data_test.dropna()

In [15]:
age_column = np.array(data_test['Age'])
age_column_temp = np.array(age_column)

In [17]:
age_column_temp

array([ 8,  1,  9,  8,  6,  1, 12,  5,  1, 11,  1,  8,  8,  8,  7,  5, 11,
        8,  8,  8,  7,  1,  8,  6,  7,  7,  9,  7,  1,  6, 11,  8,  1,  1,
        5,  9,  7,  6,  7,  9,  9,  1,  9,  6,  1,  9,  7,  8,  8,  7,  6,
        8,  8,  5, 12,  6,  7,  4,  4,  8,  1,  8,  8,  4,  9,  7,  8,  6,
        1,  7,  6,  5,  7,  9,  7,  6,  4,  9,  7,  7,  5,  8,  1,  5,  6,
        5,  5,  8,  1,  6,  6,  1,  6,  8,  5,  8,  8,  5,  1,  7,  5,  8,
        9,  8,  8,  8,  1,  7,  7,  7,  6,  9,  6,  7,  8,  5,  6,  5,  8,
        7,  9,  5,  9,  4,  8,  1,  8,  7,  1,  6,  6,  9,  9,  7,  8,  7,
        8,  1,  7,  9,  6,  7,  7,  6,  5,  9,  6,  7,  6,  7,  6,  9,  8,
        5,  1,  6,  9,  8,  7,  6,  1,  7,  8,  7,  8,  7,  7,  6,  8,  7,
        6,  9,  7,  6,  8,  7,  7,  6,  4,  6,  8,  8,  7,  7,  9,  7,  5,
        4,  7,  7,  6,  8,  7,  8,  6,  6,  6,  6,  7,  8,  9,  7,  6,  6,
        7,  9,  8,  9,  6,  7,  7,  7,  1,  7,  7,  6,  8,  7,  8,  8,  5,
        8,  7,  7,  6,  8

In [21]:
#convert age categories into age in years
age_column[age_column==0] = 23
age_column[age_column==1] = 27
age_column[age_column==2] = 32
age_column[age_column==3] = 37
age_column[age_column==4] = 42
age_column[age_column==5] = 47
age_column[age_column==6] = 52
age_column[age_column==7] = 57
age_column[age_column==8] = 62
age_column[age_column==9] = 67
age_column[age_column==10] = 72
age_column[age_column==11] = 77
age_column[age_column==12] = 82

In [23]:
data_test['Age']=age_column

In [27]:
X_test, Y_test = data_test[feat_gbt_without_log], data_test.iloc[:,-1]

In [28]:
X_test['TIME_LAPSE_LOG'] = np.log(X_test['TIME_LAPSE'])

c:\users\simay\appdata\local\programs\python\python36\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [29]:
X_test = X_test.drop(columns=['TIME_LAPSE'])

In [31]:
#X_train, Y_train = X_train.values, Y_train.values
X_test, Y_test = X_test.values, Y_test.values


In [32]:
if num_class==2:
    Y_train[Y_train==2] = 1
    Y_test[Y_test==2] = 1

In [33]:
np.unique(Y_train, return_counts=True)

(array([0, 1, 2], dtype=int64), array([485, 142, 259], dtype=int64))

In [34]:
# gradient boosting tree
if estimator_type == 'gbt':
    from sklearn.ensemble import GradientBoostingClassifier
    params = {'learning_rate': 0.1, 'max_depth': 2, 'n_estimators': 70}
    estimator = GradientBoostingClassifier(**params)
else:
    from sklearn.tree import DecisionTreeClassifier
    params = {'max_leaf_nodes': 13, 'criterion':'gini'}
    estimator = DecisionTreeClassifier(**params)
print(estimator)


GradientBoostingClassifier(criterion='friedman_mse', init=None,
              learning_rate=0.1, loss='deviance', max_depth=2,
              max_features=None, max_leaf_nodes=None,
              min_impurity_decrease=0.0, min_impurity_split=None,
              min_samples_leaf=1, min_samples_split=2,
              min_weight_fraction_leaf=0.0, n_estimators=70,
              n_iter_no_change=None, presort='auto', random_state=None,
              subsample=1.0, tol=0.0001, validation_fraction=0.1,
              verbose=0, warm_start=False)


In [35]:
np.unique(Y_test, return_counts=True)

(array([0, 1], dtype=int64), array([137, 169], dtype=int64))

<b>Cross-Validation </b>

In [36]:
random_state = 320
shuffle= True

from sklearn.model_selection import StratifiedKFold, RepeatedStratifiedKFold, cross_validate

def cross_validate_custom(X, y, num_repeated, estimator):
    n_splits = 8
    if num_repeated > 1:
        print("num_repeated is not 1")
        skf = RepeatedStratifiedKFold(n_splits=n_splits, n_repeats=num_repeated, random_state=random_state)
    else:
        skf = StratifiedKFold(n_splits=n_splits, shuffle=shuffle, random_state=random_state)
        #     scores = cross_validate(estimator, X, y, scoring='accuracy', n_jobs=-1, cv=skf, verbose=0, 
        #                             return_estimator=True, return_train_score=True)
    test_scores = []
    CM = np.zeros((num_class,num_class))
    sensitivities = []
    sepcificities = []
    for train_index, test_index in skf.split(X, y):
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]
        estimator.fit(X_train, y_train, sample_weight=[w[i] for i in y_train])
        score = estimator.score(X_test, y_test, sample_weight=[w[i] for i in y_test])
        test_scores.append(score)
        cm = confusion_matrix(y_test, estimator.predict(X_test))
        CM += cm
        if num_class == 2:
            tn, fp, fn, tp = cm.ravel() 
            TPR = tp/ (tp+fn)
            FPR = fp/(fp+tn)
            sensitivities.append(TPR)
            sepcificities.append(1 - FPR)
    # return np.mean(scores['test_score']), np.mean(scores['train_score'])
    # print(test_scores)
    return test_scores, sensitivities, sepcificities, CM 


In [37]:
test_scores, sensitivities, sepcificities, CM  = cross_validate_custom(X_train, Y_train, 5, estimator)
print('Cross-Validation')
print('accu')
print('mean:', np.mean(test_scores))
print('std:', np.std(test_scores))
print('sensitivity')
print('mean:', np.mean(sensitivities))
print('std:', np.std(sensitivities))
print('sepcificity')
print('mean:', np.mean(sepcificities))
print('std:', np.std(sepcificities))
print('CM:')
print(CM)

num_repeated is not 1
Cross-Validation
accu
mean: 0.917498184779052
std: 0.0316359284423817
sensitivity
mean: nan
std: nan
sepcificity
mean: nan
std: nan
CM:
[[2320.   94.   11.]
 [  26.  656.   28.]
 [   8.  158. 1129.]]


c:\users\simay\appdata\local\programs\python\python36\lib\site-packages\numpy\core\fromnumeric.py:3257: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
c:\users\simay\appdata\local\programs\python\python36\lib\site-packages\numpy\core\_methods.py:161: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
c:\users\simay\appdata\local\programs\python\python36\lib\site-packages\numpy\core\_methods.py:217: RuntimeWarning: Degrees of freedom <= 0 for slice
  keepdims=keepdims)
c:\users\simay\appdata\local\programs\python\python36\lib\site-packages\numpy\core\_methods.py:186: RuntimeWarning: invalid value encountered in true_divide
  arrmean, rcount, out=arrmean, casting='unsafe', subok=False)
c:\users\simay\appdata\local\programs\python\python36\lib\site-packages\numpy\core\_methods.py:209: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


In [38]:
_, counts = np.unique(Y_test, return_counts=True)
counts

array([137, 169], dtype=int64)

In [41]:
counts * w

ValueError: operands could not be broadcast together with shapes (2,) (3,) 

**Train and test using feature subset**

In [42]:
print(f'X train: {X_train.shape}')
print(f'Y train: {len(Y_train)}')
print(f'X test: {X_test.shape}')
print(f'Y test: {len(Y_test)}')

X train: (886, 8)
Y train: 886
X test: (306, 8)
Y test: 306


In [43]:
estimator= estimator.fit(X_train, Y_train, sample_weight=[w[i] for i in Y_train])
train_accu = estimator.score(X_train, Y_train, sample_weight=[w[i] for i in Y_train])
print('train_accu:')
print(train_accu)
feature_weight_pair = sorted(zip(feats, estimator.feature_importances_), key=lambda pair : pair[1], reverse=True)
print('learnt importance:')
for pair in feature_weight_pair:
    print(f"{pair[0]}")
    print(f"{pair[1]:.6f}")

train_accu:
0.9538384300515261
learnt importance:
ArmSwelling
0.445784
SYM_COUNT
0.423758
BreastSwelling
0.082886
TIME_LAPSE_LOG
0.036973
Age
0.006641
FHT
0.002084
Skin
0.001340
Mobility
0.000534


In [ ]:
'''accu = estimator.score(X_test, Y_test)
print(f'accuracy: {accu}')
CM = confusion_matrix(Y_test, estimator.predict(X_test))
print('confusion matrix:')
print(CM)
if num_class == 2:
    tn, fp, fn, tp = CM.ravel() 
    TPR = tp/ (tp+fn)
    FPR = fp/(fp+tn)
    print('sensitivity', TPR)
    print('sepcifivity', 1 - FPR)
    
    '''
    

In [44]:
w

[0.15913545, 0.54254953, 0.29831502]

<b> TEST ON WEB DATA </b>

In [45]:
Y_test_probs = estimator.predict_proba(X_test)
Y_test_2_clinical_label = np.copy(Y_test)


In [46]:
print(Y_test_2_clinical_label.shape)
np.unique(Y_test_2_clinical_label)

(306,)


array([0, 1], dtype=int64)

In [47]:
t1 = 0.490
label_t1 = ((Y_test_probs[:,1]+Y_test_probs[:,2])> t1).astype(int)

In [48]:
conf_matrix = confusion_matrix(Y_test_2_clinical_label, label_t1)

In [49]:
conf_matrix

array([[ 49,  88],
       [  5, 164]], dtype=int64)

In [51]:
tn, fp, fn, tp = conf_matrix.ravel() 
TPR = tp/ (tp+fn)
FPR = fp/(fp+tn)
accu = (tp + tn) / (tn + fp + fn + tp)

In [52]:
print(TPR, FPR, accu)

0.9704142011834319 0.6423357664233577 0.696078431372549


NOTE: I DID NOT RUN AFTER THIS CELL

<b> Calculate T1 </b>

In [ ]:
from sklearn.metrics import roc_curve
from sklearn.metrics import confusion_matrix
from sklearn import metrics

import matplotlib.pyplot as plt

In [ ]:
def t_eval_fold(t ,y_ts, yprob, get_t1=True):
    '''
    assumes y_ts has 3 class classification
    '''
    #get probabilities
    if get_t1:
        print('T1')
        ypred_current = ((yprob[:,1]+yprob[:,2])>t).astype(int)
    else:
        print('T2')
        ypred_current = ((yprob[:,2] / (yprob[:,1] + yprob[:,2])) > t).astype(int) + 1
        
        
    #print('y_ts_3class', y_ts)
    #print('y_ts_2class', y_ts)
    
    # print('ypred',ypred_current )
    print(confusion_matrix(y_ts, ypred_current))
    tn, fp, fn, tp = confusion_matrix(y_ts, ypred_current).ravel()
    TPR = tp/ (tp+fn)
    FPR = fp/(fp+tn)
    
    accu = (tp + tn) / (tn + fp + fn + tp)
   
    #returns senstivity and specifity
    return TPR , 1- FPR, accu, tn, fp, fn, tp

def thresholding_with_folds(Y_true, Y_prob, get_t1=True):
    T_range = 1.005
    t_values = list(np.arange(0.0,T_range,0.005))
    t_values = [round(elem,3) for elem in t_values]
    averages = []
    for t in t_values:
        sens_list =[]
        spec_list =[]
        accu_list = []
        kf = RepeatedStratifiedKFold(n_splits=8, n_repeats=5,random_state=1)
        # kf = KFold(n_splits=8 ,shuffle= False, random_state= True )
        for _, test_index in kf.split(Y_prob, Y_true): #for each fold
            # print("TRAIN:", train_index, "TEST:", test_index)
            sens, spec, accu, _, _, _, _ = t_eval_fold(t, Y_true[test_index], Y_prob[test_index], get_t1)
            sens_list.append(sens)
            spec_list.append(spec)
            accu_list.append(accu)
        sens_avg = np.mean(sens_list)
        sens_std= np.std(sens_list)
        spec_avg = np.mean(spec_list)
        spec_std =np.std(spec_list)
        accu_avg = np.mean(accu_list)
        accu_std = np.std(accu_list)
        averages.append([t, sens_avg , sens_std, spec_avg, spec_std, accu_avg, accu_std])
    return averages


def draw_roc_curve(result_matrix, title):
    fpr_list= []
    tpr_list= []
    for tup in result_matrix:
        t = tup[0]
        senstivity , specifity, accu = tup[1] , tup[3], tup[5]
        tpr_list.append(senstivity)
        fpr_list.append(1-specifity)
        print(t ,'\t' ,senstivity, '\t'  ,1-specifity, '\t', accu)
    plt.figure(figsize=(12,8))
    plt.plot(fpr_list, tpr_list)
    plt.title(title)
    plt.ylabel('TPR')
    plt.xlabel('FPR')
    plt.show()
    
def test_t(t, test_2_class_labels, test_probs, title, get_t1=True):
    result = thresholding_with_folds(test_2_class_labels, test_probs, get_t1)
    draw_roc_curve(result, title)
    TPR , TNR, accu, tn, fp, fn, tp = t_eval_fold(t, test_2_class_labels, test_probs, get_t1)
    print('TPR: ', TPR)
    print('TNR: ', TNR)
    print('accu: ', accu)
    print('CM')
    print(np.array([[tn, fp],[fn, tp]]))

In [ ]:
Y_train_prob = estimator.predict_proba(X_train)
Y_train_2_class = np.copy(Y_train)
Y_train_2_class[Y_train_2_class==2] = 1
print(Y_train_prob.shape)

In [ ]:
#A = list(zip(Y_train_prob[:,0], Y_train))
#A = [x for x in A if x[1] in [1,2]]
#print('max p1 for label 1 or 2 samples')
#np.max(np.array(A)[:,0],axis=0)

In [ ]:
#t = 0.49
#ypred_current = ((Y_train_prob[:,1]+Y_train_prob[:,2])>t).astype(int)

In [ ]:
#confusion_matrix(Y_train_2_class, ypred_current)

In [ ]:
result = thresholding_with_folds(Y_train_2_class, Y_train_prob)

In [ ]:
result

In [ ]:
draw_roc_curve(result, 'ROC Curve of t1 for Decision Tree on Training Set')

In [ ]:
# test t 
Y_test_probs = estimator.predict_proba(X_test)
Y_test_2_class_label = np.copy(Y_test)
Y_test_2_class_label[Y_test_2_class_label==2] = 1
print(Y_test_2_class_label.shape)
np.unique(Y_test_2_class_label)

In [ ]:
t = 0.49
test_t(t, Y_test_2_class_label, Y_test_probs, 'ROC Curve of t1 for Decision Tree on Test Set')

Find t2

In [ ]:
Y_train_2_class = np.copy(Y_train)
Y_train_2_class = Y_train_2_class[~(Y_train==0)]
Y_train_2_prob = Y_train_prob[~(Y_train==0)]
print(Y_train_2_prob.shape)

In [ ]:
result = thresholding_with_folds(Y_train_2_class, Y_train_2_prob, get_t1=False)

In [ ]:
result

In [ ]:
draw_roc_curve(result, 'ROC Curve of t2 for Decision Tree on Training Set')

Test t2

In [ ]:
Y_test_2_class_label = np.copy(Y_test)
Y_test_2_class_label[Y_test_2_class_label==2] = 1

In [ ]:
t1 = 0.490
label_t1 = ((Y_test_probs[:,1]+Y_test_probs[:,2])> t1).astype(int)
#label_t1 = ((Y_test_probs[:,0]/(Y_test_probs[:,1]+Y_test_probs[:,2] + 0.001))<t1).astype(int)
confusion_matrix(Y_test_2_class_label, label_t1)


In [ ]:
Y_test_probs_pass_t1 = Y_test_probs[(label_t1 == 1) & ~(Y_test == 0)]
Y_test_pass_t1 = Y_test[(label_t1 == 1) & ~(Y_test == 0)]
print(Y_test_pass_t1.shape)

In [ ]:
t2 = 0.295
#ypred_t2 = ((Y_test_probs_pass_t1[:,1]/(Y_test_probs_pass_t1[:,2] + 0.001)<t2) + 1).astype(int)
test_t(t2, Y_test_pass_t1, Y_test_probs_pass_t1, 'ROC Curve of t2 for Decision Tree on Test Set', get_t1=False)

**Train and test using all feature**

In [ ]:
X_train = data.values
X_test = data_test.values
print(f'X train: {X_train.shape}')
print(f'Y train: {len(Y_train)}')
print(f'X test: {X_test.shape}')
print(f'Y test: {len(Y_test)}')

In [ ]:
params = {'learning_rate': 0.1, 'max_depth': 2, 'n_estimators': 70}
gbt = GradientBoostingClassifier(**params)
gbt= gbt.fit(X_train, Y_train)
feature_weight_pair = sorted(zip(data.columns.values, gbt.feature_importances_), key=lambda pair : pair[1], reverse=True)
print('learnt importance:')
feature_weight_pair

In [ ]:
accu = gbt.score(X_test, Y_test, sample_weight=[w[i] for i in Y_test])
print(f'accuracy: {accu}')
CM = confusion_matrix(Y_test, gbt.predict(X_test))
print('confusion matrix:')
print(CM)

Check if there are sample overlap between the train and validation:

In [ ]:
dataTest = pd.read_csv(test_set_dir)
dataTrain = pd.read_csv(DATA_PATH)

In [ ]:
s1 = pd.merge(dataTest, dataTrain, how='inner', on=['Username'])

In [ ]:
s1

In [ ]:
# print(sorted(dataTest['Username'].values))

In [ ]:
# print(sorted(dataTrain['Username'].values))

Check label distribution

In [ ]:
{label:count for label, count in zip(*np.unique(Y_test, return_counts=True))}

In [ ]:
{label:count for label, count in zip(*np.unique(Y_train, return_counts=True))}

In [ ]:
a = np.array([583, 171, 311])

In [ ]:
w = a / np.sum(a)

In [ ]:
w = w[0] / w

In [ ]:
w / np.sum(w)

In [ ]:
paris = [("Arm Swelling", 0.456441),
("Symptom Count", 0.421406),
("BreastSwelling", 0.073692),
("TIME LAPSE", 0.037715), 
("Age", 0.006301),
("FHT", 0.002236),
("Fibrosis", 0.001830),
("Mobility", 0.000378)]


In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
feat_names, feat_scores = list(zip(*paris[::-1]))

In [ ]:
f, ax = plt.subplots(figsize=(15, 6))
ax.barh(feat_names, feat_scores, align='center')
ax.set_xlabel('importance score')
ax.set_title('importance scores leant by gradient boosting trees')
ax.set_xlim([0, 0.5])
for i, v in enumerate(feat_scores):
    ax.text(v, i, str(v), color='blue', fontweight='bold')